In [15]:
import sys
import numpy as np
import random
from pathlib import Path
import yaml
import secrets
import xarray as xr
import matplotlib.pyplot as plt
# import seaborn as sns
# from plotssrc import pltsds, pltmoms, animations
from pySD.plotssrc import pltsds, pltmoms, animations
from pySD.sdmout_src import *
from pySD.sdmout_src import sdtracing
from pySD.gbxboundariesbinary_src import read_gbxboundaries as rgrid
from pySD.initsuperdropsbinary_src import *
from pySD.initsuperdropsbinary_src import read_initsuperdrops as rsupers
from pySD.thermobinary_src import read_thermodynamics as rthermo


from sdm_eurec4a.visulization import set_custom_rcParams

set_custom_rcParams()

path2CLEO = Path("/home/m/m301096/CLEO")
sys.path.append(path2CLEO)  # for imports from pySD package
sys.path.append(path2CLEO / "examples/exampleplotting") # for imports from example plotting package
# from plotssrc import pltsds, pltmoms, animations

ModuleNotFoundError: No module named 'seaborn'

In [16]:
path2build = path2CLEO / "build"
configfile = path2CLEO / "eurec4a/experiment_02/src/config/rain1d_config.txt"

# yaml_config_file = sys.argv[1]
yaml_config_file = Path("/home/m/m301096/") / "repositories/sdm-eurec4a/data/model/input/example_input_18.yaml"
with open(yaml_config_file, 'r') as f:
    config_yaml = yaml.safe_load(f)


### ---------------------------------------------------------------- ###
### ----------------------- INPUT PARAMETERS ----------------------- ###
### ---------------------------------------------------------------- ###
### --- essential paths and filenames --- ###
# path and filenames for creating initial SD conditions
constsfile    = path2CLEO / "libs/cleoconstants.hpp"
binpath       = path2build / "bin/"
sharepath     = path2build / "share/"
gridfile      = sharepath / "rain1d_dimlessGBxboundaries.dat"
initSDsfile   = sharepath / "rain1d_dimlessSDsinit.dat"
thermofile    =  sharepath / "rain1d_dimlessthermo.dat"

# path and file names for plotting results
setupfile     = path2CLEO / "data/output/raw/rain1d_setup.txt"
dataset       = path2CLEO / "data/output/raw/rain1d_sol.zarr"

### --- plotting initialisation figures --- ###
isfigures   = [True, True] # booleans for [making, saving] initialisation figures
cloud_id = config_yaml['cloud']['cloud_id']
identification_type = config_yaml['cloud']['identification_type']
savefigpath = path2CLEO / "results/experiment_02" / f"{identification_type}_{cloud_id}" # directory for saving figures
savefigpath.mkdir(exist_ok=True, parents=True)

In [47]:
# read in constants and intial setup from setup .txt file
config = pysetuptxt.get_config(setupfile, nattrs=3, isprint=False)
consts = pysetuptxt.get_consts(setupfile, isprint=False)

# Create a first simple dataset to have the coordinates for later netcdf creation
sddata = pyzarr.get_supers(str(dataset), consts)
simple_ds = xr.open_dataset(dataset, engine="zarr",
                                consolidated=False);
config

---- Superdrop Properties -----
RHO_L = 998.203 Kg/m^3
RHO_SOL = 2016.5 Kg/m^3
MR_SOL = 0.05844277 Kg/mol
IONIC = 2.0
-------------------------------
supers dataset:  /home/m/m301096/CLEO/data/output/raw/rain1d_sol.zarr


{'maxchunk': 1250000.0,
 'nspacedims': 1,
 'ngbxs': 60.0,
 'totnsupers': 5120.0,
 'CONDTSTEP': 1.0,
 'COLLTSTEP': 1.0,
 'MOTIONTSTEP': 2.0,
 'COUPLTSTEP': 720.0,
 'OBSTSTEP': 5.0,
 'T_END': 720.0,
 'cond_iters': 2.0,
 'cond_SUBTSTEP': 0.1,
 'cond_rtol': 0.0,
 'cond_atol': 0.01,
 'numSDattrs': 4,
 'ntime': 145}

## Use the preprocessed ``full_dataset``


To do so, we create a list of temporaray filenames

In [18]:
path2CLEO = Path("/home/m/m301096/CLEO")
path2dataoutput = path2CLEO / "data/output"
TEMPORARY_DIR = path2dataoutput / "temporary"
OUTPUT_DIR = path2dataoutput / "processed/" f"{identification_type}_{cloud_id}"

### Load CLEO result

In [34]:
ds_full = xr.open_dataset(OUTPUT_DIR / "full_dataset.nc")
ds_full["mass"] = 4/3 * np.pi * ds_full["radius"]**3


# Select relevant droplets
- needs to leave domain at one point

In [36]:
ds = ds_full.where(ds_full.coord3.min("time") <= 40, drop=True)
ds

<xarray.Dataset>
Dimensions:     (time: 145, sd_id: 1303)
Coordinates:
  * time        (time) float64 0.0 5.0 10.0 15.0 ... 705.0 710.0 715.0 720.0
  * sd_id       (sd_id) int64 10 186 187 188 189 ... 5115 5116 5117 5118 5119
Data variables:
    xi          (time, sd_id) float64 50.0 8.444e+04 9.135e+04 ... nan nan nan
    radius      (time, sd_id) float64 0.1453 81.5 85.41 88.16 ... nan nan nan
    coord3      (time, sd_id) float64 803.5 806.5 809.4 807.3 ... nan nan nan
    sdgbxindex  (time, sd_id) float64 40.0 40.0 40.0 40.0 ... nan nan nan nan
    msol        (time, sd_id) float64 2.589e-14 4.573e-06 5.263e-06 ... nan nan
    mass        (time, sd_id) float64 0.01284 2.268e+06 2.61e+06 ... nan nan nan

In [22]:
ds['mass_diff'] = ds.mass.diff(dim = "time")

In [28]:
ds.coord3

<xarray.DataArray 'coord3' (time: 145, sd_id: 5120)>
array([[ 800.090221,  818.58881 ,  800.077523, ..., 1199.682046, 1197.148831,
        1181.029032],
       [ 800.090176,  818.58876 ,  800.077472, ..., 1162.953571, 1159.5574  ,
        1142.385429],
       [ 800.090135,  818.588715,  800.077425, ..., 1138.455866, 1134.485299,
        1116.612905],
       ...,
       [ 800.08293 ,  818.580791,  800.069238, ...,         nan,         nan,
                nan],
       [ 800.082868,  818.580723,  800.069168, ...,         nan,         nan,
                nan],
       [ 800.082827,  818.580678,  800.069121, ...,         nan,         nan,
                nan]])
Coordinates:
  * time     (time) float64 0.0 5.0 10.0 15.0 20.0 ... 705.0 710.0 715.0 720.0
  * sd_id    (sd_id) int64 0 1 2 3 4 5 6 ... 5113 5114 5115 5116 5117 5118 5119

In [31]:
from sdm_eurec4a.reductions import shape_dim_as_dataarray

time = shape_dim_as_dataarray(ds.coord3, output_dim='time')
sd_id = shape_dim_as_dataarray(ds.coord3, output_dim='sd_id')

plt.scatter(
    time,
    ds.coord3,
    ds.mass,
    # cmap = "spring",
    # shading="nearest"
    )
# plt.colorbar()



Error in callback <function flush_figures at 0x7fff9fee9800> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
# Plot the values
ax.scatter(
    ds.isel(time = 10).radius, 
    ds.isel(time = 10).xi,
    c = ds.isel(time = 10).mass,
    marker='o')
ax.set_xscale('log')
ax.set_yscale('log')